## 安裝套件

In [1]:
!pip install torch-geometric -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.6 MB/s eta 0:00:00


## 匯入套件 & 檢查環境

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

print("PyTorch version:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

PyTorch version: 2.9.0+cu126
Using device: cuda


## 載入 Cora 節點分類資料集

In [3]:
# 下載 / 載入 Cora 資料集
dataset = Planetoid(root="./data/Cora", name="Cora")
data = dataset[0].to(device)

print(data)
print("Number of nodes:", data.num_nodes)
print("Number of node features:", data.num_node_features)
print("Number of classes:", dataset.num_classes)

Processing...
Done!


Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of node features: 1433
Number of classes: 7


## 定義一個簡單 GNN 模型

In [8]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        # TODO
        # 第一層圖卷積
        self.conv1 = GCNConv(in_channels, hidden_channels)
        # 第二層圖卷積（輸出為類別數）
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout_rate = dropout


    def forward(self, x, edge_index):
        # TODO
        # 第一層：GCNConv + ReLU
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # 第二層：GCNConv
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


model = GNN(
    in_channels=dataset.num_node_features,
    hidden_channels=16,
    out_channels=dataset.num_classes,
    dropout=0.5,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# 損失函數（Loss Function)(Negative Log Likelihood Loss, 負對數似然損失)
# 常用於多類別分類（Multi-Class Classification），差異越大，損失值越高。
criterion = nn.NLLLoss()

print(model)

GNN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)


## 訓練 & 測試迴圈

In [9]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    # 只針對 train 節點算 loss
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()


@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

    # 分別計算 train/val/test 的 accuracy
    accs = []
    for mask_name, mask in [
        ("train", data.train_mask),
        ("val", data.val_mask),
        ("test", data.test_mask),
    ]:
        correct = (pred[mask] == data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)
    return accs  # [train_acc, val_acc, test_acc]

## 實際訓練模型

In [20]:
best_val_acc = 0.0
best_test_acc = 0.0

for epoch in range(1, 100):  # 100 epochs
    loss = train()
    train_acc, val_acc, test_acc = test()

    # 追蹤 valdation 最好的時候的 test acc
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc

    if epoch % 10 == 0 or epoch == 1:
        print(
            f"Epoch: {epoch:03d} | "
            f"Loss: {loss:.4f} | "
            f"Train Acc: {train_acc:.3f} | "
            f"Val Acc: {val_acc:.3f} | "
            f"Test Acc: {test_acc:.3f}"
        )

print(f"\nBest Val Acc: {best_val_acc:.3f}, Test Acc at that time: {best_test_acc:.3f}")

Epoch: 001 | Loss: 0.0168 | Train Acc: 1.000 | Val Acc: 0.768 | Test Acc: 0.804
Epoch: 010 | Loss: 0.0245 | Train Acc: 1.000 | Val Acc: 0.770 | Test Acc: 0.805
Epoch: 020 | Loss: 0.0181 | Train Acc: 1.000 | Val Acc: 0.766 | Test Acc: 0.793
Epoch: 030 | Loss: 0.0204 | Train Acc: 1.000 | Val Acc: 0.784 | Test Acc: 0.815
Epoch: 040 | Loss: 0.0207 | Train Acc: 1.000 | Val Acc: 0.774 | Test Acc: 0.797
Epoch: 050 | Loss: 0.0255 | Train Acc: 1.000 | Val Acc: 0.752 | Test Acc: 0.803
Epoch: 060 | Loss: 0.0161 | Train Acc: 1.000 | Val Acc: 0.774 | Test Acc: 0.812
Epoch: 070 | Loss: 0.0144 | Train Acc: 1.000 | Val Acc: 0.776 | Test Acc: 0.803
Epoch: 080 | Loss: 0.0199 | Train Acc: 1.000 | Val Acc: 0.770 | Test Acc: 0.808
Epoch: 090 | Loss: 0.0211 | Train Acc: 1.000 | Val Acc: 0.760 | Test Acc: 0.794

Best Val Acc: 0.784, Test Acc at that time: 0.815
